In [1]:
import logging
import datetime
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import collections
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import config
import modules.dataloaders as data_loaders
import modules.utils as utils
import modules.models as models
import modules.loss as loss_module
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch

# AIMET imports

In [2]:
from decimal import Decimal

from aimet_torch.compress import ModelCompressor
from aimet_torch.defs import ChannelPruningParameters
from aimet_torch.onnx_utils import OnnxSaver
from aimet_common.defs import CostMetric, CompressionScheme, GreedySelectionParameters

2024-07-20 02:19:29,621 - root - INFO - AIMET


# Supress Warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Define Matplot Style

In [4]:
#mpl.style.use('seaborn-v0_8')

# Logger

In [5]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Detector.\n' +  
            '\tDFire and FASDD UAV and CV.\n' +
            '\tFASDD: train and val datasets to train, and test dataset to validate.\n' +
            '\tFASDD RS not included, as it only has smoke and it is too different to current pictures\n' + 
            f'\tPruning Compression Ratio  = {config.PRUNING_COMPRESSION_RATIO}\n')

# Hyperparameters Log

In [6]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('IMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nGrid, Bounding Boxes, Classes, Max Obj and Thresholds:')
print(f'\tGrid: {config.S}')
print(f'\tNumber of Bounding Boxes per Cell: {config.B}')
print(f'\tNumber of Classes: {config.C}')
print(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
print(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
print(f'\tScore Threshold: {config.SCORE_THRESHOLD}')
print('\nAIMET Configuration')
print(f'\tUse Previous Dic: {config.USE_PREVIOUS_DIC}')
print(f'\tSpatial SVD Compression: {config.SVD_COMPRESSION_RATIO}')
print(f'\tPrunning Compression: {config.PRUNING_COMPRESSION_RATIO}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('IMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nGrid, Bounding Boxes, Classes and Thresholds:')
logger.info(f'\tGrid: {config.S}')
logger.info(f'\tNumber of Bounding Boxes per Cell: {config.B}')
logger.info(f'\tNumber of Classes: {config.C}')
logger.info(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
logger.info(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
logger.info(f'\tScore Threshold: {config.SCORE_THRESHOLD}\n')
logger.info('\nAIMET Configuration')
logger.info(f'\tUse Previous Dic: {config.USE_PREVIOUS_DIC}')
logger.info(f'\tSpatial SVD Compression: {config.SVD_COMPRESSION_RATIO}')
logger.info(f'\tPrunning Compression: {config.PRUNING_COMPRESSION_RATIO}')


Datasets Length
	Train: Full
	Val: Full

Load Model: True
	Model: ./experiments/test_31_svd_080_simple_model/weights/BED_detector__best_mAP=0.6204__epoch=4.pt
Device: cuda
Optimizer:
	Learning Rate: 0.0005
	Gradients Clip Norm: 500
	Weight Decay: 0.0005
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 1
	Scheduler threshold: 0.01
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 40
IMG DIMS:
	Width: 224
	Height: 224

Grid, Bounding Boxes, Classes, Max Obj and Thresholds:
	Grid: 7
	Number of Bounding Boxes per Cell: 2
	Number of Classes: 2
	Maximum Number of Objects per Image: 10
	IOU Threshold: 0.5
	Score Threshold: 0.2

AIMET Configuration
	Use Previous Dic: True
	Spatial SVD Compression: 0.8
	Prunning Compression: 0.9


# Dataset Setup

In [7]:
train_loader = data_loaders.get_train_loader()

# val_loader = data_loaders.get_val_loader(
#     dfire_len = 900,
#     fasdd_uav_len = 900,
#     fasdd_cv_len = 3400)
val_loader = data_loaders.get_val_loader()


TRAIN DFIRE dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


DFire Removed wrong images: 0
DFire Removed due to overlapping: 1292
DFire Removed due to more than 10: 59

Train DFire dataset len: 15870

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 1233
FASDD Removed due to more than 10: 449

Train FASDD UAV dataset len: 10869

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 841
FASDD Removed due to more than 10: 300

Val FASDD UAV dataset len: 7224

TRAIN FASDD CV dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 2141
FASDD Removed due to more than 10: 342

Train FASDD CV dataset len: 45177

VAL FASDD CV dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 1238
FASDD Removed due to more than 10: 221

Val FASDD CV dataset len: 30311

Concatenate Train DFire and Train FASDD UAV datasets
Train dataset len: 26739
Concatenate with Val FASDD UAV dataset
Train dataset len: 33963
Concatenate with Train FASDD CV dataset
Train dataset len: 79140
Concatenate with Val FASDD CV dataset
Train dataset len: 109451

TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Val FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test d

# Plot Some Train Pictures

In [8]:
for batch_idx, (img, label) in enumerate(train_loader):
       
    if batch_idx == 0:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 224, 224])


# Plot Some Val Pictures

In [9]:
for batch_idx, (img, label) in enumerate(val_loader):
       
    if batch_idx == 27:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 224, 224])


# Loss Setup

In [10]:
if config.LOSS_FN == "YOLOV1_LOSS":
    print(f'Loss Function: YOLOV1_LOSS')
    logger.info(f'\nLoss Function: YOLOV1_LOSS')
    loss_fn = loss_module.YoloLoss_2BBox()
    print(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
    logger.info(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: YOLOV1_LOSS
Lambda for L1 regularization: 0


# Model Setup

In [11]:
if config.MODEL == "BED":
    
    print("Using BED Detector")
    logger.info("\nUsing BED Detector")
    model = models.SVD_BED_DETECTOR().to(config.DEVICE)  

else:
    print("Wrong Model")
    logger.info("Wrong Model")
    raise SystemExit("Wrong Model")

# optimizer = optim.Adam(model.parameters(), 
#                        lr=config.LEARNING_RATE, 
#                        weight_decay=config.WEIGHT_DECAY)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
#                                                  mode='min',
#                                                  factor=config.FACTOR, 
#                                                  patience=config.PATIENCE, 
#                                                  threshold=config.THRES, 
#                                                  threshold_mode='abs',
#                                                  min_lr=config.MIN_LR)

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}')

Using BED Detector

Trainable parameters = 229209
Total parameters = 229209



### Check Model Shape

In [12]:
in_rand_np = np.random.rand(4, 3, config.IMG_H, config.IMG_W)
in_rand = torch.tensor(in_rand_np, dtype=torch.float32, device=config.DEVICE)
out_test = model(in_rand)

print(f'Input shape is {in_rand.shape}')
print(f'Model shape is {out_test.shape}')
print(f'BED Model Arquitecture\n{model}')
logger.info(f'\nInput shape is {in_rand.shape}')
logger.info(f'Model shape is {out_test.shape}\n')
logger.info(f'BED Model Arquitecture\n{model}')

Input shape is torch.Size([4, 3, 224, 224])
Model shape is torch.Size([4, 12, 7, 7])
BED Model Arquitecture
SVD_BED_DETECTOR(
  (model): Sequential(
    (conv1): Sequential(
      (0): Conv2d(3, 5, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(5, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (dropout1): Dropout2d(p=0.3, inplace=False)
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Sequential(
      (0): Conv2d(32, 6, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(6, 16, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (dropout2): Dropout2d(p=0.3, inplace=False)
    (maxpool3): MaxPool2d(ker

### Torch Summary

In [13]:
print(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))
logger.info("\nModel Summary")
logger.info(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
SVD_BED_DETECTOR                         [1, 12, 7, 7]             --
├─Sequential: 1-1                        [1, 12, 7, 7]             --
│    └─Sequential: 2-1                   [1, 32, 224, 224]         --
│    │    └─Conv2d: 3-1                  [1, 5, 224, 224]          45
│    │    └─Conv2d: 3-2                  [1, 32, 224, 224]         480
│    └─BatchNorm2d: 2-2                  [1, 32, 224, 224]         64
│    └─ReLU: 2-3                         [1, 32, 224, 224]         --
│    └─Dropout2d: 2-4                    [1, 32, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 32, 112, 112]         --
│    └─Sequential: 2-6                   [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-3                  [1, 6, 112, 112]          576
│    │    └─Conv2d: 3-4                  [1, 16, 112, 112]         288
│    └─BatchNorm2d: 2-7                  [1, 16, 112, 112]         32
│    └─ReLU:

# Load Pretrained or Initialize Weights

In [14]:
epochs_trained = utils.load_checkpoint(config.LOAD_MODEL_FILE, 
                                       model, 
                                       optimizer=None, 
                                       scheduler=None, 
                                       device=config.DEVICE)

logger.info(f"Loading Model. Trained during {epochs_trained} epochs")

Loading Model. Trained during 4 epochs


# AIMET Pruning

### Configure Pruning Parameters

In [15]:
modules_to_ignore = [model.model.conv1[0], model.model.conv72, model.model.conv73, model.model.conv74]
greedy_params = GreedySelectionParameters(target_comp_ratio=Decimal(config.PRUNING_COMPRESSION_RATIO),
                                          saved_eval_scores_dict=config.PRUNING_DIC_FILE)
auto_params = ChannelPruningParameters.AutoModeParams(greedy_params,
                                                  modules_to_ignore=modules_to_ignore)
cp_params = ChannelPruningParameters(mode=ChannelPruningParameters.Mode.auto,
                                     params=auto_params,
                                     data_loader=val_loader,
                                     num_reconstruction_samples=500,
                                     allow_custom_downsample_ops=False)

### Evaluate Model Callback

Signature: (model, iterations, use_cuda)
Return an accuracy metric

In [16]:
def evaluate_model(model, iterations, use_cuda):
   
    model.eval()
    
    for batch_idx, (x, y) in enumerate(val_loader):
        if use_cuda == True:
            x, y = x.to('cuda'), y.to('cuda')
        else:
            model.to('cpu')
        out = model(x)

        # Remove Permute from the model
        out = out.permute(0, 2, 3, 1)
        
        if iterations is not None:
            if batch_idx == iterations:
                break

        # Mean Average Precision
        for idx in range(x.shape[0]):
            target_boxes = metrics.get_true_boxes(y[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(out[idx].detach().to('cpu'))
            metrics.map_metric.update(preds = pred_boxes, target = target_boxes) 

    meanAP = metrics.map_metric.compute()
    metrics.map_metric.reset()
    print(f'Val mAP = {meanAP["map_50"]:.4f}')
    
    return meanAP['map_50'].item()

### Baseline mAP Mean

In [17]:
baseline_mAP = evaluate_model(model, None, True)
print(type(baseline_mAP))

logger.info(f'Baseline mAP: {baseline_mAP}')

Val mAP = 0.6186
<class 'float'>


### Input Shape

In [18]:
input_shape = (1, 3, config.IMG_H, config.IMG_W)

In [19]:
comp_model, stats = ModelCompressor.compress_model(model,
                                                   input_shape=input_shape,
                                                   eval_callback=evaluate_model,
                                                   eval_iterations=None,
                                                   compress_scheme=CompressionScheme.channel_pruning,
                                                   cost_metric=CostMetric.memory,
                                                   parameters=cp_params,
                                                   visualization_url=None) 

2024-07-20 02:37:47,529 - CompRatioSelect - INFO - Greedy selection: Read eval dict from ./data/greedy_selection_eval_scores_dict.pkl
2024-07-20 02:37:47,530 - CompRatioSelect - INFO - Greedy selection: overall_min_score=0.000000, overall_max_score=0.624938
2024-07-20 02:37:47,531 - CompRatioSelect - INFO - Greedy selection: Original model cost=(Cost: memory=227149, mac=143663296)
2024-07-20 02:38:28,824 - CompRatioSelect - INFO - Greedy selection: final choice - comp_ratio=0.890482, score=0.618339
2024-07-20 02:38:30,475 - ChannelPruning - INFO - finished linear regression fit 
2024-07-20 02:38:32,095 - ChannelPruning - INFO - finished linear regression fit 
2024-07-20 02:38:33,731 - ChannelPruning - INFO - finished linear regression fit 
2024-07-20 02:38:35,397 - ChannelPruning - INFO - finished linear regression fit 
2024-07-20 02:38:37,095 - ChannelPruning - INFO - finished linear regression fit 
2024-07-20 02:38:38,765 - ChannelPruning - INFO - finished linear regression fit 
2024

In [20]:
print(comp_model)
logger.info(comp_model)

SVD_BED_DETECTOR(
  (model): Sequential(
    (conv1): Sequential(
      (0): Conv2d(3, 5, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(5, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (dropout1): Dropout2d(p=0.3, inplace=False)
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Sequential(
      (0): Conv2d(32, 4, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(4, 11, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn2): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (dropout2): Dropout2d(p=0.3, inplace=False)
    (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv31): Sequential(
      (0): Conv2d(11

### Print Stats

In [21]:
print(stats)
logger.info(stats)

**********************************************************************************************
Compressed Model Statistics
Baseline model accuracy: 0.618582, Compressed model accuracy: 0.288772
Compression ratio for memory=0.890482, mac=0.744655

**********************************************************************************************

Per-layer Stats
    Name:model.conv1.1, compression-ratio: None
    Name:model.conv2.0, compression-ratio: None
    Name:model.conv2.1, compression-ratio: 0.7
    Name:model.conv31.0, compression-ratio: 0.7
    Name:model.conv31.1, compression-ratio: None
    Name:model.conv32.0, compression-ratio: 0.9
    Name:model.conv32.1, compression-ratio: 0.9
    Name:model.conv33, compression-ratio: 0.8
    Name:model.conv34.0, compression-ratio: 0.7
    Name:model.conv34.1, compression-ratio: 0.6
    Name:model.conv41, compression-ratio: None
    Name:model.conv42.0, compression-ratio: 0.9
    Name:model.conv42.1, compression-ratio: None
    Name:model.conv

### Torchinfo: model compressed summary

In [22]:
print(summary(comp_model, input_size=(1, 3, config.IMG_H, config.IMG_W)))
logger.info("Compressed Model Summary")
logger.info(summary(comp_model, input_size=(1, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
SVD_BED_DETECTOR                         [1, 12, 7, 7]             --
├─Sequential: 1-1                        [1, 12, 7, 7]             --
│    └─Sequential: 2-1                   [1, 32, 224, 224]         --
│    │    └─Conv2d: 3-1                  [1, 5, 224, 224]          45
│    │    └─Conv2d: 3-2                  [1, 32, 224, 224]         480
│    └─BatchNorm2d: 2-2                  [1, 32, 224, 224]         64
│    └─ReLU: 2-3                         [1, 32, 224, 224]         --
│    └─Dropout2d: 2-4                    [1, 32, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 32, 112, 112]         --
│    └─Sequential: 2-6                   [1, 11, 112, 112]         --
│    │    └─Conv2d: 3-3                  [1, 4, 112, 112]          384
│    │    └─Conv2d: 3-4                  [1, 11, 112, 112]         132
│    └─BatchNorm2d: 2-7                  [1, 11, 112, 112]         22
│    └─ReLU:

### Evaluate Compressed Model

In [23]:
comp_mAP = evaluate_model(comp_model, None, True)

Val mAP = 0.2888


### Visualize Results

In [24]:
comp_ratios_file_path = './data/greedy_selection_comp_ratios_list.pkl'
eval_scores_path = './data/greedy_selection_eval_scores_dict.pkl'

unpickled_ratios = pd.read_pickle(comp_ratios_file_path)
unpickled_scores = pd.read_pickle(eval_scores_path)

In [25]:
df_scores = pd.DataFrame(unpickled_scores)
df_scores.to_csv(config.RUN_FOLDER + 'scores.csv')
print(df_scores)

     model.conv1.1  model.conv2.0  model.conv2.1  model.conv31.0  \
0.1       0.257727       0.262289       0.266260        0.012131   
0.2       0.259994       0.436184       0.268609        0.342664   
0.3       0.260303       0.560970       0.266243        0.340688   
0.4       0.608439       0.608733       0.595722        0.614254   
0.5       0.601097       0.606773       0.615182        0.618928   
0.6       0.617922       0.613751       0.611310        0.618132   
0.7       0.618788       0.615676       0.620876        0.622012   
0.8       0.617812       0.615529       0.620346        0.618489   
0.9       0.617101       0.616634       0.622989        0.621017   

     model.conv31.1  model.conv32.0  model.conv32.1  model.conv33  \
0.1        0.145364        0.137725        0.112070      0.198202   
0.2        0.141169        0.601511        0.525943      0.389893   
0.3        0.134961        0.602791        0.547771      0.516458   
0.4        0.140416        0.618732        

In [26]:
df_ratios = pd.DataFrame(unpickled_ratios)
df_ratios.to_csv(config.RUN_FOLDER + 'ratios.csv')
print(df_ratios)

                 0     1
0    model.conv1.1  None
1    model.conv2.0  None
2    model.conv2.1   0.7
3   model.conv31.0   0.7
4   model.conv31.1  None
5   model.conv32.0   0.9
6   model.conv32.1   0.9
7     model.conv33   0.8
8   model.conv34.0   0.7
9   model.conv34.1   0.6
10    model.conv41  None
11  model.conv42.0   0.9
12  model.conv42.1  None
13    model.conv43  None
14  model.conv44.0   0.9
15  model.conv44.1   0.8
16  model.conv45.0   0.9
17  model.conv45.1   0.9
18  model.conv46.0   0.9
19  model.conv46.1   0.9
20  model.conv51.0   0.9
21  model.conv51.1  None
22  model.conv52.0  None
23  model.conv52.1   0.8
24    model.conv53  None
25  model.conv54.0  None
26  model.conv54.1   0.8
27  model.conv55.0  None
28  model.conv55.1  None
29  model.conv56.0  None
30  model.conv56.1   0.9
31    model.conv61  None
32    model.conv62  None
33    model.conv71  None


# Save Compressed Before Training

In [27]:
torch.onnx.export(comp_model, torch.randn(input_shape).to(config.DEVICE), config.RUN_FOLDER + 'comp_model_noTrain.onnx')

### Optimizer and Scheduler of Compressed Model to Train

In [28]:
optimizer = optim.Adam(comp_model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)

utils.save_checkpoint(epoch=0, 
                      model=comp_model,
                      optimizer=optimizer,
                      scheduler=scheduler,
                      checkpoint_name=config.WEIGHTS_FOLDER + 'comp_model_after_svd.pt')

# Loss and Metrics Loggers and Plotters

In [29]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Train Loop Function

In [37]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_mAP = torch.tensor(0., dtype=torch.float32)

    epochs_loss_plot = []
    epochs_metric_plot = []

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
            
        calculate_mAP = False
        if ( (epoch+1) % 5 ) == 0:
            calculate_mAP = True
            epochs_metric_plot.append(epoch)

        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            loss_l1_lambda=config.LAMBDA_L1_LOSS,
            metric=metrics.map_metric,
            device=config.DEVICE,
            calculate_mAP=calculate_mAP)
        
        train_losses_logger.update_losses(train_losses)
        if calculate_mAP == True:
            train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN STATS", train_losses, train_metrics, mAP_available=calculate_mAP))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                metric=metrics.map_metric,
                device=config.DEVICE,
                calculate_mAP=calculate_mAP)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_losses(val_losses)
            if calculate_mAP == True:
                val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL STATS", val_losses, val_metrics, mAP_available=calculate_mAP))
            
        epochs_loss_plot.append(epoch)

        # loss_plotter.plot_all_metrics(
        #     train_losses_logger.get_losses(),
        #     val_losses_logger.get_losses(),
        #     epochs_loss_plot)

        # if calculate_mAP == True:
        #     metrics_plotter.plot_all_metrics(
        #         train_metrics_logger.get_metrics(),
        #         val_metrics_logger.get_metrics(),
        #         epochs_metric_plot)

        # lr_logger.plot_lr(epochs_loss_plot)
        
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.3f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.3f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  

        # Save model if mAP increases
        if calculate_mAP == True:
            if ( best_mAP < val_metrics['mAP'] ) :
                best_mAP = val_metrics['mAP']
                print(f"\nSaving model with new best mAP: {best_mAP:.4f}")
                logger.info(f"Saving model with new best mAP: {best_mAP:.4f}")
                save_precision_name = f'best_mAP={best_mAP:.4f}__epoch={epoch}'
                save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + save_precision_name + '.pt'
                utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
            # Break if model is good enough
            if (best_mAP.item() > 0.65):
                break
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_detector.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

# Training

In [31]:
print("Starting script\n")
logger.info("Starting script\n")
    
pruned_model = train_loop(comp_model)

Starting script


***Start Training: 02:41:02


=== EPOCH 0/39 ===
Learning Rate = 0.0005



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [03:54<00:00,  7.29it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
71.389      |25.382      |33.412      |6.872       |5.723       


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:31<00:00, 11.42it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
61.756      |23.293      |24.663      |8.953       |4.847       

Saving model with new best validation loss: 61.756

=== EPOCH 1/39 ===
Learning Rate = 0.0005



...

Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
49.217      |17.645      |20.756      |6.727       |4.089       

=== EPOCH 39/39 ===
Learning Rate = 6.710886400000004e-05



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [03:56<00:00,  7.22it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
45.780      |14.845      |21.951      |6.974       |2.010       
Train mAP = 0.5685


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:33<00:00, 10.64it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
49.183      |17.665      |20.581      |6.831       |4.106       
Val mAP = 0.6264

Saving model with new best validation loss: 49.183

Saving model with new best mAP: 0.6264

***Script finished: 05:42:03

Time elapsed: 3:01:01.751338


# Check Comp Model Params

In [35]:
# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in comp_model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = sum(p.numel() for p in comp_model.parameters())
print(f'Total parameters = {n_params}')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 204232
Total parameters = 204232


In [33]:
comp_model_mAP = evaluate_model(comp_model, None, True)

Val mAP = 0.6264


# Export to ONNX

In [34]:
torch.onnx.export(comp_model, torch.randn(input_shape).to(config.DEVICE), config.RUN_FOLDER + 'comp_model_Train.onnx')

# Train More

In [38]:
pruned_model = train_loop(
    comp_model,
    start_epoch=40,
    epochs_to_train=15)


***Start Training: 05:53:55


=== EPOCH 40/54 ===
Learning Rate = 6.710886400000004e-05



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [03:52<00:00,  7.35it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
45.488      |14.748      |21.778      |6.966       |1.997       


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:32<00:00, 11.05it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
49.206      |17.594      |20.634      |6.852       |4.126       

Saving model with new best validation loss: 49.206

=== EPOCH 41/54 ===
Learning Rate = 5.3687091200000036e-05



...

Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
48.759      |17.487      |20.520      |6.648       |4.105       

=== EPOCH 54/54 ===
Learning Rate = 1.7592186044416018e-05



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [03:54<00:00,  7.28it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
44.624      |14.311      |21.494      |6.908       |1.911       
Train mAP = 0.5747


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:33<00:00, 10.76it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
48.774      |17.432      |20.510      |6.750       |4.082       
Val mAP = 0.6268

Saving model with new best mAP: 0.6268

***Script finished: 07:02:11

Time elapsed: 1:08:16.263945
